# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [100]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import gmaps.datasets
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [101]:
df = pd.read_csv('WeatherData.csv')
df.head()

,index,lat,lng,city,country,temp,humidity,cloudiness,windspeed,max_temp
0,0,6.275321,-117.464170,san patricio,mx,84.2,66,40,11.41,84.2
1,1,-15.476659,-165.247211,alofi,nu,84.2,79,50,6.93,84.2
2,3,-80.334241,-83.362865,ushuaia,ar,50.0,71,75,18.34,50.0
3,4,61.502872,94.802000,baykit,ru,-2.7,99,83,2.95,-2.7
4,5,-42.611450,-79.412777,castro,cl,64.4,63,20,5.82,64.4


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [102]:
gmaps.configure(api_key=gkey)

In [103]:
# Create dataframe with necessary data
coords = df[['lat','lng']]
humidity = df['humidity'].astype(float)
maxHumidity = humidity.max()

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(coords, weights=humidity,dissipating=False,max_intensity=maxHumidity,point_radius=1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [104]:
# Filter value to meet criteria
# Changed criteria slightly to increase number of cities
df2 = df[df['max_temp'] < 80]
df2 = df2[df2['max_temp'] > 70]
df2 = df2[df2['windspeed'] < 20]
df2 = df2[df2['cloudiness'] < 20]

df2

,index,lat,lng,city,country,temp,humidity,cloudiness,windspeed,max_temp
8,9,-22.683293,-130.195217,rikitea,pf,79.34,78,0,11.39,79.34
87,141,6.817340,-106.813759,ixtapa,mx,78.89,65,5,10.29,78.80
92,151,-39.543369,-26.286587,arraial do cabo,br,77.00,80,0,10.11,75.70
120,213,-13.502331,111.645898,boyolangu,id,77.74,84,14,2.73,77.74
124,221,-27.748838,-44.451621,sao sebastiao,br,76.50,76,0,16.28,76.50
135,246,7.330248,104.526126,ca mau,vn,71.91,89,7,2.86,71.91
140,262,-23.538769,20.292460,aranos,na,70.07,15,0,12.59,70.07
147,284,-31.854882,143.399969,broken hill,au,78.80,31,0,12.75,78.80
177,359,6.909411,-24.745691,sao filipe,cv,73.71,67,0,10.74,73.71
221,467,-35.647849,177.408278,tairua,nz,78.01,72,19,7.00,77.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [105]:
hotel_df = df2
hotel_df['Hotel Name'] = ''

In [106]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

params = {
    'key' : gkey,
    'radius' : 10000,
    'type' : 'lodging'
}

for index, row in hotel_df.iterrows():
    lat = row['lat']
    lng = row['lng']
    
    params['location'] = f'{lat},{lng}'
     
    try:
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
    except (IndexError):
        hotel_df.loc[index,'Hotel Name'] = np.nan
        
hotel_df


,index,lat,lng,city,country,temp,humidity,cloudiness,windspeed,max_temp,Hotel Name
8,9,-22.683293,-130.195217,rikitea,pf,79.34,78,0,11.39,79.34,NaN
87,141,6.817340,-106.813759,ixtapa,mx,78.89,65,5,10.29,78.80,NaN
92,151,-39.543369,-26.286587,arraial do cabo,br,77.00,80,0,10.11,75.70,NaN
120,213,-13.502331,111.645898,boyolangu,id,77.74,84,14,2.73,77.74,NaN
124,221,-27.748838,-44.451621,sao sebastiao,br,76.50,76,0,16.28,76.50,NaN
135,246,7.330248,104.526126,ca mau,vn,71.91,89,7,2.86,71.91,NaN
140,262,-23.538769,20.292460,aranos,na,70.07,15,0,12.59,70.07,NaN
147,284,-31.854882,143.399969,broken hill,au,78.80,31,0,12.75,78.80,NaN
177,359,6.909411,-24.745691,sao filipe,cv,73.71,67,0,10.74,73.71,NaN
221,467,-35.647849,177.408278,tairua,nz,78.01,72,19,7.00,77.00,NaN


In [107]:
# Remove nulls from Hotel_DF

hotel_df2 = hotel_df.dropna()
hotel_df2

,index,lat,lng,city,country,temp,humidity,cloudiness,windspeed,max_temp,Hotel Name
373,899,28.48235,-97.889309,beeville,us,74.93,57,1,3.36,78.01,Casa Bonita Ranch


In [109]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df2.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [110]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))